In [0]:
with cte_data (id, x, y, xy, pt) as
(
select monotonically_increasing_id()
     , split_part(value, ',', 1)::int as x
     , split_part(value, ',', 2)::int as y
     , x + y / 100000::decimal(14,7)
     , ST_Point(x, y)
  from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_09_input.txt', format => 'text', wholeText => false) as t (value)
/*from values ('7,1')
            , ('11,1')
            , ('11,7')
            , ('9,7')
            , ('9,5')
            , ('2,5')
            , ('2,3')
            , ('7,3') as t (value)*/
)
  ,  cte_polygon (arg, poly) as
(
select array_sort(array_agg(struct(id, pt)), (l, r) -> (l.id > r.id)::int * 2 - 1) as arg
     , st_makepolygon(st_makeline(array_append(arg.pt, arg[0].pt)))
  from cte_data
)
-- This approach is more performant because computing the st_contains is where most of the time geos
-- Running the operation on the sorted array make it stops computing as soon as the largest area is found.
select max((abs(t2.x - t1.x) + 1l)::bigint * (abs(t2.y - t1.y)::bigint + 1l)) as part1
     , aggregate( array_sort( array_agg(struct((abs(t2.x - t1.x) + 1l)::bigint * (abs(t2.y - t1.y)::bigint + 1l) as a, array(t1.pt, t2.pt) as pts))
                            , (l, r) -> (l.a < r.a)::int * 2 - 1)
                , null::bigint
                , (agg, p) -> coalesce(agg, case when st_contains(any_value(poly), st_envelope(st_makeline(p.pts))) then p.a end)
                ) as part2
  from cte_data as t1
  join cte_data as t2 on t1.xy < t2.xy
  join cte_polygon    on true;

